## Introduction

This is mostly a "fun" excercise using topic modelling (more precisely Latent Dirichlet Allocation, LDA)  and Self Organizing Maps (SOMs).  

It is not my intention to explain LDA here. There are a wealth of resources out there for that. One that I particularly like is [this tutorial](https://cran.r-project.org/web/packages/topicmodels/vignettes/topicmodels.pdf) of the R package `topicmodels`. Even if you don't code in R, the writting is very nice and clear. For example, taken directly from that document: the generative model consists of the following three steps. 

1. The term distribution $\beta$ is determined for each topic by: 

    $\beta \sim$ Dirichlet($\delta$)

    $\beta$ is the term distribution of topics and contains the probability of a word occurring
    in a given topic

2. The proportions $\theta$ of the topic distribution for the document $w$ are determined by

    $\theta \sim$  Dirichlet($\alpha$).

3. For each of the N words $w_i$

    (a) Choose a topic $z_i \sim$ Multinomial($\theta$).
    
    (b) Choose a word $w_i$ from a multinomial probability distribution conditioned on the topic $z_i$: 
    
    $p(w_i|z_i, \beta)$.


So...in (perhaps too) "simpler" words. We are trying to "reconstruct" a document based on the most probable words associated to the topics present in that document. All this thorugh an EM algorithm.

For this excercise I will use the well known and easily tractable [20newsgroup](http://www.cs.cmu.edu/afs/cs.cmu.edu/project/theo-20/www/data/news20.html) dataset. In the "near" (we'll see how near...) future we intend to do something more sophisticated with a [larger dataset](https://github.com/fabianmurariu/website-categories-nn) with content for nearly a million urls. 

Anyway, let's start.

In [1]:
import os,sys,re
from bs4 import BeautifulSoup

# wherever you place your dataset
TEXT_DATA_DIR = '/home/ubuntu/working/text_classification/20_newsgroup/'

docs = []
doc_classes = []
for name in sorted(os.listdir(TEXT_DATA_DIR)):
    path = os.path.join(TEXT_DATA_DIR, name)
    if os.path.isdir(path):
        for fname in sorted(os.listdir(path)):
            if fname.isdigit():
                fpath = os.path.join(path, fname)
                f = open(fpath)
                t = f.read()
                # skip header
                i = t.find('\n\n')
                if 0 < i:
                    t = t[i:]
                t = BeautifulSoup(t).get_text()
                t = re.sub("[^a-zA-Z]"," ", t)
                docs.append(t)
                doc_classes.append(name)
                f.close()

/home/ubuntu/anaconda2/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 174 of the file /home/ubuntu/anaconda2/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [2]:
# let's print the first ~1000 characters of the 1st document
print(docs[0][:1001])

Archive name  atheism resources Alt atheism archive name  resources Last modified     December      Version                                     Atheist Resources                        Addresses of Atheist Organizations                                       USA  FREEDOM FROM RELIGION FOUNDATION  Darwin fish bumper stickers and assorted other atheist paraphernalia are available from the Freedom From Religion Foundation in the US   Write to   FFRF  P O  Box      Madison  WI        Telephone                  EVOLUTION DESIGNS  Evolution Designs sell the  Darwin fish    It s a fish symbol  like the ones Christians stick on their cars  but with feet and the word  Darwin  written inside   The deluxe moulded  D plastic fish is       postpaid in the US   Write to   Evolution Designs       Laurel Canyon     North Hollywood             CA         People in the San Francisco Bay area can get Darwin Fish from Lynn Gold    try mailing    For net people who go to Lynn directly  the price is       pe

## EXPERIMENTS

We are going to run 3 experiments applying different preprocessing routines to the documents before passing them to the LDA model. We will evaluate each set up using the probability-based metric [perplexity](http://qpleple.com/perplexity-to-evaluate-topic-models/) (I will come back to this metric later). In the `lda_perplexity.py` script the experiments are wrapped up in functions like this
    
```
def experiment_1(docs, tokenizer_):
    train_docs, test_docs = train_test_split(docs, test_size=0.25, random_state=0)
    vectorizer = CountVectorizer(min_df=10, max_df=0.5,max_features=MAX_NB_WORDS,tokenizer = tokenizer_)
    return pipeline(train_docs, test_docs, vectorizer, lda_model)

#to run
basic_pp_exp1, basic_tw_exp1 = experiment_1(docs, SimpleTokenizer)
stem_pp_exp1, stem_tw_exp1 = experiment_1(docs, StemTokenizer())
lemma_pp_exp1, lemma_tw_exp1 = experiment_1(docs, LemmaTokenizer())
```
   
In this notebook I will describe in detail experiment_1 and I will go faster through experiment_2 and 3.

## Experiment 1: CountVectorizer with different tokenizers

Here I will simply use 3 tokenizers: a simple tokenizer (lowercases, tokenizes and removes stopwords), a "StemTokenizer" which applies stemization to the documents, and a "LemmaTokenizer" which applies lemmatization. Normally, one would use Lemmatization because the results are easier to interpret. This is the code for the tokenizers:

In [3]:
from nltk.stem import WordNetLemmatizer, PorterStemmer
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

def SimpleTokenizer(doc):
    """Basic tokenizer using gensim's simple_preprocess

    Parameters:
    ----------
    docs (list): list of documents

    Returns:
    ----------
    tokenized documents
    """
    return [t for t in simple_preprocess(doc, min_len=3) if t not in STOPWORDS]


class StemTokenizer(object):
    """Stem tokens in a document

    Parameters:
    ----------
    docs (list): list of documents

    Returns:
    --------
    list of stemmed tokens
    """
    def __init__(self):
        self.stemmer = PorterStemmer()
    def __call__(self, doc):
        return [self.stemmer.stem(t) for t in SimpleTokenizer(doc)]


class LemmaTokenizer(object):
    """Lemmatize tokens in a document

    Parameters:
    ----------
    docs (list): list of documents

    Returns:
    --------
    list of lemmatized tokens
    """
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.lemmatizer.lemmatize(t, pos="v") for t in SimpleTokenizer(doc)]

In the repo these classes are in the `nlp_utils.py` scrip and are imported from there

Now let's define our model:

In [4]:
from sklearn.decomposition import LatentDirichletAllocation

# Lda model that will be used through all the experiments
NB_TOPICS = 10
lda_model = LatentDirichletAllocation(
    n_components=NB_TOPICS,
    learning_method='online',
    max_iter=10,
    batch_size=2000,
    verbose=1,
    max_doc_update_iter=100,
    n_jobs=-1,
    random_state=0)

Some might be saying: *"wait, sklearn LDA? why not the implemenation at the well known and wonderfull gensim package".* Fair question, you can go and have a look to the scrip `LDA_gensim_vs_sklearn.ipynb` or for more information. 

And now, this is how the experiment goes:

In [5]:
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

# 0-train/test split
MAX_NB_WORDS = 20000
train_docs, test_docs = train_test_split(docs, test_size=0.25, random_state=0)
vectorizer = CountVectorizer(min_df=10, max_df=0.5, max_features=MAX_NB_WORDS, tokenizer = LemmaTokenizer())

# 1-vectorize
tr_corpus = vectorizer.fit_transform(train_docs)
te_corpus = vectorizer.transform(test_docs)
n_words = len(vectorizer.vocabulary_)

# 2-train model
model = lda_model.fit(tr_corpus)

# 3-compute perplexity
gamma = model.transform(te_corpus)
perplexity = model.perplexity(te_corpus, gamma)/n_words

# 4-get vocabulary and return top N words. Let's 1st define a little helper
def get_topic_words(topic_model, feature_names, n_top_words):
    """Helper to get n_top_words per topic

    Parameters:
    ----------
    topic_model: LDA model
    feature_names: vocabulary
    n_top_words: number of top words to retrieve

    Returns:
    -------
    topics: list of tuples with topic index, the most probable words and the scores/probs
    """
    topics = []
    for topic_idx, topic in enumerate(topic_model.components_):
        topic_words = [feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
        tot_score = np.sum(topic)
        scores = [topic[i]/tot_score for i in topic.argsort()[:-n_top_words - 1:-1]]
        topics.append([topic_idx, zip(topic_words, scores)])
    return topics


features = vectorizer.get_feature_names()
top_words = get_topic_words(model,features,10)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:812: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if doc_topic_distr != 'deprecated':
/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:816: DeprecationWarning: Argument 'doc_topic_distr' is deprecated and is being ignored as of 0.19. Support for this argument will be removed in 0.21.
  DeprecationWarning)


let's have a look to the results:

In [8]:
print("Perplexity of the LDA model with Lemmatization-preprocessing: {}".format(perplexity))
top_words

Perplexity of the LDA model with Lemmatization-preprocessing: 0.266592870347


[[0,
  [(u'edu', 0.0067918785580433324),
   (u'article', 0.0067569641886425745),
   (u'think', 0.0065112375141850448),
   (u'people', 0.0062470350411229641),
   (u'know', 0.0052563821487041345),
   (u'time', 0.0050749510534372905),
   (u'like', 0.0048143805076149727),
   (u'com', 0.0045819950601443246),
   (u'post', 0.0038861505854044542),
   (u'use', 0.0038454649715609765)]],
 [1,
  [(u'edu', 0.010447325839301027),
   (u'article', 0.0079042426298446666),
   (u'like', 0.0074978798166720423),
   (u'com', 0.0074467939943020254),
   (u'think', 0.0067673233063638314),
   (u'car', 0.0064237856975015864),
   (u'go', 0.0058587958527275708),
   (u'know', 0.0054723931238338875),
   (u'get', 0.0047341178454663214),
   (u'time', 0.004604247531864154)]],
 [2,
  [(u'space', 0.010835665044824305),
   (u'israel', 0.0088573078069718418),
   (u'nasa', 0.006539412054020137),
   (u'israeli', 0.0053793226519720716),
   (u'jews', 0.0053435897725169776),
   (u'launch', 0.0052689609705854658),
   (u'new', 0.

Regarding to the `perplexity`, if all the words in our vocabulary would occur with equal probability, then the perplexity would be 1.0. It would be really hard to, given some words, associate them with a topic. In other words, in terms of finding topics, we would be really confused, perplexed! Therefore, the lower the perplexity, the better the model. 

However, [Chang et al., 2009](https://www.umiacs.umd.edu/~jbg/docs/nips2009-rtl.pdf) showed that perplexity is not always correlated with topic intrepretability (and when you are doing topic modelling, you might want your topics to be as interpretable as possible). In this context [Röder et al, 2015](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) proposed a pipeline to estimate the so called `topic Coherence` that proved to be better correlated with human generated topic rankings. The `Coherence` metric is implemented in gensim, and I found another "didactic" implementation [here](https://github.com/jhlau/topic_interpretability). 

In a broad brush sense, there are different ways of estimating the `Coherence`, but are all based on probabilities of word co-occurrence. The way I understand the process described in their paper is as follows: 

1. Segmentation of a set into a smaller sets (e.g word pairs) 
2. Calculate the so-called Confirmation Measures that score the agreement of a given pair (e.g. the pointwise mutual information (PMI) or the normalized PMI (NPMI)). Confirmation Measures use word and word co-occurrence probailities (see [Röder et al, 2015](http://svn.aksw.org/papers/2015/WSDM_Topic_Evaluation/public.pdf) their expression (1) and (2) for example)
3. Aggregation of the Confirmation Measures

If we refer to the set of segmentations as $S$, the set of confirmation meassures as $M$, the set of word probabilities as $P$ and the aggregations functions as $\Sigma$, Röder and collaborators define their coherence pipeline as $C = S \times M \times P \times \Sigma$

I experimented a bit with Coherence and that can be found in `LDA_coherence_demo.ipynb`.

Now let's go back to our code/experiments. I have run experiment_1 with a `LemmaTokenizer`. The same can be done with the `SimpleTokenizer` and `StemTokenizer` by simply changing the `tokenizer` parameter in `CountVectorizer`. For now, let's move to the 2nd experiment.

## Experiment 2: CountVectorizer with different tokenizers and bigrams

The code here is mostly identical, we just need to introduce a new class

In [9]:
from gensim.models.phrases import Phraser, Phrases

class Bigram(object):
    """Bigrams to get phrases like artificial_intelligence

    Parameters:
    ----------
    docs (list): list of documents

    Returns:
    --------
    the document with bigrams appended at the end
    """
    def __init__(self):
        self.phraser = Phraser
    def __call__(self, docs):
        phrases = Phrases(docs,min_count=20)
        bigram = self.phraser(phrases)
        for idx in range(len(docs)):
            for token in bigram[docs[idx]]:
                if '_' in token:
                    docs[idx].append(token)
        return docs

From here on, using again `LemmaTokenizer`: 

In [10]:
vectorizer = CountVectorizer(
    min_df=10, max_df=0.5,
    max_features=MAX_NB_WORDS,
    preprocessor = lambda x: x,
    tokenizer = lambda x: x)
tokenizer_ = LemmaTokenizer()
tokens = [tokenizer_(doc) for doc in docs]
phraser_ = Bigram()
ptokens = phraser_(tokens)

# 0-train/test split
train_docs, test_docs = train_test_split(ptokens, test_size=0.25, random_state=0)

# 1-vectorize
tr_corpus = vectorizer.fit_transform(train_docs)
te_corpus = vectorizer.transform(test_docs)
n_words = len(vectorizer.vocabulary_)

# 2-train model
model = lda_model.fit(tr_corpus)

# 3-compute perplexity
gamma = model.transform(te_corpus)
perplexity = model.perplexity(te_corpus, gamma)/n_words

# 4-get vocabulary and return top N words.
features = vectorizer.get_feature_names()
top_words = get_topic_words(model,features,10)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:816: DeprecationWarning: Argument 'doc_topic_distr' is deprecated and is being ignored as of 0.19. Support for this argument will be removed in 0.21.
  DeprecationWarning)


In [11]:
print(perplexity)
top_words

0.258999126731


[[0,
  [(u'file', 0.014911280157244338),
   (u'image', 0.010087368004218188),
   (u'edu', 0.009198718391388392),
   (u'program', 0.0086550028771869639),
   (u'use', 0.0075488499161402078),
   (u'window', 0.0058757980358187844),
   (u'graphics', 0.0054010947879012941),
   (u'windows', 0.0050675226372621073),
   (u'include', 0.0050662931002265869),
   (u'display', 0.0047945041399675989)]],
 [1,
  [(u'people', 0.007344598485807553),
   (u'state', 0.0059917967001673874),
   (u'right', 0.0056321028217693097),
   (u'say', 0.0044455979349985386),
   (u'government', 0.0044290789455164407),
   (u'think', 0.0042957762398633916),
   (u'israel', 0.0041167481937032209),
   (u'president', 0.0038552756959020014),
   (u'time', 0.003481948886547852),
   (u'work', 0.0033615996187893503)]],
 [2,
  [(u'use', 0.0068769583314697205),
   (u'space', 0.0067398018754097223),
   (u'chip', 0.0059586126831805168),
   (u'key', 0.005910233879818063),
   (u'scsi', 0.0047015105484066629),
   (u'article', 0.00461100083

Similar results but slightly lower perplexity. And finally

## Experiment 3: Filtering words

Here we will use a vocabulary to filter some words. To my experience, these approaches do not work well (lead to higher perplexities) but...worth trying. 

In [12]:
from nltk.corpus import words

class WordFilter(object):
    """Filter words based on a vocabulary

    Parameters:
    ----------
    vocab: the vocabulary used for filtering
    doc  : the document containing the tokens to be filtered

    Returns:
    -------
    filetered document
    """
    def __init__(self, vocab):
        self.filter = vocab
    def __call__(self, doc):
        return [t for t in doc if t in self.filter]


wordfilter = WordFilter(vocab=set(words.words()))
tokens = [SimpleTokenizer(doc) for doc in docs]
ftokens = [wordfilter(d) for d in tokens]

# 0-train/test split
train_docs, test_docs = train_test_split(ftokens, test_size=0.25, random_state=0)

# 1-vectorize
tr_corpus = vectorizer.fit_transform(train_docs)
te_corpus = vectorizer.transform(test_docs)
n_words = len(vectorizer.vocabulary_)

# 2-train model
model = lda_model.fit(tr_corpus)

# 3-compute perplexity
gamma = model.transform(te_corpus)
perplexity = model.perplexity(te_corpus, gamma)/n_words

# 4-get vocabulary and return top N words.
features = vectorizer.get_feature_names()
top_words = get_topic_words(model,features,10)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


/home/ubuntu/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:816: DeprecationWarning: Argument 'doc_topic_distr' is deprecated and is being ignored as of 0.19. Support for this argument will be removed in 0.21.
  DeprecationWarning)


In [14]:
print(perplexity)
top_words

0.342266878772


[[0,
  [(u'article', 0.017580034444793155),
   (u'objective', 0.013131849459992533),
   (u'science', 0.01221008169911989),
   (u'frank', 0.01086650145166858),
   (u'moral', 0.010355460503028671),
   (u'theory', 0.010050946200559904),
   (u'think', 0.009480288761605591),
   (u'people', 0.0091244576103224184),
   (u'morality', 0.0083461139872245838),
   (u'value', 0.0082145670489376515)]],
 [1,
  [(u'game', 0.022896501711834081),
   (u'team', 0.022492179496859718),
   (u'year', 0.013789294025856455),
   (u'hockey', 0.012704702048298417),
   (u'play', 0.011010993838714867),
   (u'win', 0.011007070437857393),
   (u'new', 0.010558831078797752),
   (u'season', 0.0098853327743381892),
   (u'league', 0.0083729823937593265),
   (u'pit', 0.0080520131850816464)]],
 [2,
  [(u'turkey', 0.0314575726942741),
   (u'day', 0.022628921534513388),
   (u'article', 0.014152941958146139),
   (u'sun', 0.012359529167810064),
   (u'send', 0.0088843568817670894),
   (u'law', 0.0077262802105048422),
   (u'days', 

Perplexity is worse and the topic words do not look much better. Anyway...I have run the three experiments with the 3 tokenizers and 3 different number of topics values: 10, 20 and 50. For example:  

```
python lda_perplexity.py --n_topics 10
```

The results are stored in dictionaries (pickle files). In the `LDA_coherence_demo.ipynb` notebook I have used gensim to find the coherence of the topics for the top 3 models. 